<a href="https://colab.research.google.com/github/tejask666/Tejas_INFO5731_Spring2020/blob/master/Final%20Code%20with%20comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
import os
import re,datetime
import time
import csv
from selenium.common.exceptions import NoSuchElementException
from google.colab import files
from selenium import webdriver
  
class In_Class_Assigment():
    def get_data(self,list_of_journal):
        list_of_data=[]
        print(list_of_journal)
        obj = In_Class_Assigment()
        # Launch chrome browser.
        options = webdriver.ChromeOptions()
        # Runs browser in background.
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument("--window-size=1920,1080")
        options.add_argument("start-maximized")
        options.add_argument("disable-infobars")  
        options.add_argument("--disable-extensions")
        driver = webdriver.Chrome('chromedriver',options=options)
        #This loop will run twice as there are two journals.
        for journal in range(len(list_of_journal)):
            with open(r'/content/Book1.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([list_of_journal[journal][3]])
            list_of_data = list_of_journal[journal]
            print(list_of_data)
            print(list_of_data[2])
            #driver.get is use to access url. 
            driver.get(list_of_data[2])
            try:
              if driver.find_element_by_xpath("(//button[@aria-expanded='false'])[8]").is_displayed():
                # finds year,but before that it checks it is already expanded or not.
                 driver.find_element_by_xpath("//div[text()='Date Published']").click()
                 time.sleep(5)
            except NoSuchElementException as msg:
                print("No such element found")
            driver.find_element_by_xpath("//input[@id='start-year']").clear()
            driver.find_element_by_xpath("//input[@id='start-year']").send_keys(list_of_data[0])
            driver.find_element_by_xpath("//input[@id='end-year']").clear()
            driver.find_element_by_xpath("//input[@id='end-year']").send_keys(list_of_data[1])
            driver.find_element_by_xpath("(//input[@id='end-year']//following::input[@type='submit'])[1]").click()
            time.sleep(10)
            #It find how many page are there on which articles are present.
            total_pages = driver.find_element_by_xpath("(//span[@class='number-of-pages'])[1]").text
            count = int(total_pages)
            for page in range(count-1):#count
                list_of_articles = driver.find_elements_by_xpath("//h2//a[contains(@href,'/article/')]")
             # This loop will run for the number pages i.e. on which articles are available.  
                for i in range(len(list_of_articles)):  # len(list_of_articles)
                    time.sleep(5)
                    driver.refresh()
                    time.sleep(5)
                    list_of_articles = driver.find_elements_by_xpath("//h2//a[contains(@href,'/article/')]")
            # This is use to scroll the page.
                    driver.execute_script("arguments[0].scrollIntoView();", list_of_articles[i])
                    list_of_articles[i].click()
            #This finds webelement i.e.location of references        
                    list_of_ref = driver.find_elements_by_xpath("//p[contains(@class,'c-article-references__text')]")
                    if list_of_ref != []:
                        for j in range(len(list_of_ref)):
            # Actual content i.e. data which we want to extract from webpage is saved in reference_text in            
                            reference_text = list_of_ref[j].text
                            author = obj.get_author(reference_text)
                            if author is None:
                                continue
                            author.append(list_of_data[3])
                            obj.writeintocsv(author)
                        driver.back()
                    else:
                        driver.back()
                button=driver.find_element_by_xpath("(//img[contains(@src,'/images/arrow-right.png')])[1]")
                driver.execute_script("arguments[0].scrollIntoView();", button)     
                button.click()
                time.sleep(10)

    def get_author(self, reference):

           author_detail=[]
           tapp=[]
           tapp[:0]=reference
           #Separates author name and year, as year is mentioned in () with sentences and extracting year.  
           remove="1234567890"
           temp=""
           for i in range(len(tapp)):
              if tapp[i] in remove:
                 break
              else:
                temp=temp+tapp[i]
           author_name=temp.strip(".(")
           try:
             # Finding 4 digit in reference 
              match= re.search(r'\d{4}',reference)
              #Extracting year
              date = datetime.datetime.strptime(match.group(), '%Y').date()
              year=date.year
           except:
             author_detail=None
             return author_detail
             #Spliting article and year as year is always after dot
           article_title=reference.split(").")
           if len(article_title)>2:
              try:
                title=re.search("(?P<url>https?://[^\s]+)", article_title[2]).group("url")
              except:
                list_of_sentence=article_title[2].split(".")
                title=list_of_sentence[0].strip(" ")
           else:
              if len(article_title)==1:
                try:
                  title=re.search("(?P<url>https?://[^\s]+)", article_title[2]).group("url")
                except:
                   title=article_title[0]
              else:
                list_of_sen=article_title[1].split(".")
                title=list_of_sen[0].strip(" ")    
                #checking if author or year or author name any of the detail is not available it will none the author details
           if title=="" or year=="" or author_name=="":
                author_detail=None
                return author_detail
           else:
                author_detail.append(title)
                author_detail.append(year)
                author_detail.append(author_name)
                return author_detail
                
               #writing all information in csv file
    def writeintocsv(self,ref_details):
        with open(r'/content/Book1.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                if len(ref_details)<=2:
                   pass
                else:
                   writer.writerow(["",ref_details[0],ref_details[1],str(ref_details[2]).strip('[]')])
#This checks file already exists or not on path
object = In_Class_Assigment()
isExist = os.path.exists('/content/Book1.csv')
if isExist:
         os.remove('/content/Book1.csv')
#  if file already exists it will append in that otherwise it will create new file and append data in it.         
with open(r'/content/Book1.csv', 'a+', newline='') as file:
# Object created to use csv.writer applications.  
                writer = csv.writer(file)
                writer.writerow(["Journal Names","Article Title","Year","Author"]) 
data_list=[("1937", "2019","https://link.springer.com/search?query=&search-within=Journal&facet-journal-id=40732","Psychological Record")]
object.get_data(data_list)